In [1]:
import sys
sys.path.append("..")
from docopt import docopt
import logging
import neptune
import numpy as np
import syft as sy
from torch import load
from random import seed, sample
from torch import optim, float32, int64, tensor
from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils
CONFIG_PATH = '../configs/defaults.yml'

configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
seed(configs['runtime']['random_seed'])

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
# Logging initialization
log_enable = False
output_dir = None
neptune_enable = False

epochs_num = configs['runtime']['epochs']
rounds_num = configs['runtime']['rounds']

fl = FederatedLearning(
    configs['runtime']['batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    configs['runtime']['reg'],
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])

fl.create_server()
fl.create_server_model()


raw_train_data = utils.preprocess_leaf_data(
    utils.load_leaf_train(configs['data']['FEMNIST_PATH'])
)
raw_test_data = utils.preprocess_leaf_data(
    utils.load_leaf_test(configs['data']['FEMNIST_PATH']), min_num_samples=configs['runtime']['batch_size']
)


workers_idx = list(raw_train_data.keys())
total_num_workers = len(workers_idx)

fl.create_workers(workers_idx)
fl.create_workers_model(workers_idx)

# trusted_idx = utils.get_workers_idx(
#     range(total_num_workers), configs['runtime']['mnist_trusted_users_num'], [])
eavesdroppers_idx = utils.get_workers_idx(
    workers_idx, configs['runtime']['femnist_eavesdropper_num'], [])
normal_idx = utils.get_workers_idx(
    workers_idx, 
    total_num_workers - int(configs['runtime']['femnist_eavesdropper_num']),
    eavesdroppers_idx)

# logging.info("Trusted: {}".format(trusted_idx))
logging.info("Eavesdroppers: {}".format(eavesdroppers_idx))
logging.info("Normal: {}".format(normal_idx))


2020-12-10 21:52:44,852 Initializing Federated Learning class...
2020-12-10 21:52:45,616 Creating the server...
2020-12-10 21:52:45,617 Creating a model for the server...
2020-12-10 21:52:45,627 Loading train dataset from /home/ubuntu/data/leaf_non_iid/data/femnist/data
2020-12-10 21:52:45,629 Loading 1 out of 2 files...
2020-12-10 21:52:51,125 Loading 2 out of 2 files...
2020-12-10 21:52:56,123 Start processing of femnist data...
2020-12-10 21:52:57,788 Loading test dataset from /home/ubuntu/data/leaf_non_iid/data/femnist/data
2020-12-10 21:52:57,798 Loading 1 out of 2 files...
2020-12-10 21:52:58,242 Loading 2 out of 2 files...
2020-12-10 21:52:58,687 Start processing of femnist data...
2020-12-10 21:52:58,778 Creating workers...
2020-12-10 21:52:58,787 Creating a model for 100 worker(s)...
2020-12-10 21:52:59,192 Eavesdroppers: ['f0826_18', 'f0735_17', 'f0680_28', 'f0647_19', 'f0956_32']
2020-12-10 21:52:59,193 Normal: ['f0530_15', 'f0877_17', 'f0987_34', 'f0654_02', 'f0750_18', 'f0

In [3]:
test_dataloader = fl.create_femnist_server_test_dataloader(raw_test_data)

2020-12-10 21:52:59,203 Creating femnist test dataloader possibly for the server
2020-12-10 21:52:59,255 Length of Federated Dataset (Total number of records for all workers): 427


In [5]:
for n, (a, b) in enumerate(test_dataloader):
    print("{}: {}, {}".format(n, a.get().shape, b.get().shape))

0: torch.Size([10, 1, 28, 28]), torch.Size([10])
1: torch.Size([10, 1, 28, 28]), torch.Size([10])
2: torch.Size([10, 1, 28, 28]), torch.Size([10])
3: torch.Size([10, 1, 28, 28]), torch.Size([10])
4: torch.Size([10, 1, 28, 28]), torch.Size([10])
5: torch.Size([10, 1, 28, 28]), torch.Size([10])
6: torch.Size([10, 1, 28, 28]), torch.Size([10])
7: torch.Size([10, 1, 28, 28]), torch.Size([10])
8: torch.Size([10, 1, 28, 28]), torch.Size([10])
9: torch.Size([10, 1, 28, 28]), torch.Size([10])
10: torch.Size([10, 1, 28, 28]), torch.Size([10])
11: torch.Size([10, 1, 28, 28]), torch.Size([10])
12: torch.Size([10, 1, 28, 28]), torch.Size([10])
13: torch.Size([10, 1, 28, 28]), torch.Size([10])
14: torch.Size([10, 1, 28, 28]), torch.Size([10])
15: torch.Size([10, 1, 28, 28]), torch.Size([10])
16: torch.Size([10, 1, 28, 28]), torch.Size([10])
17: torch.Size([10, 1, 28, 28]), torch.Size([10])
18: torch.Size([10, 1, 28, 28]), torch.Size([10])
19: torch.Size([10, 1, 28, 28]), torch.Size([10])
20: torch.

In [6]:
utils.dataset_info(raw_test_data)

15:	9
16:	6
17:	5
18:	3
19:	3
Mean num of samples/user: 17.0
Total Samples:	427
Total Users:	26
[f0629_39]: Images: 15, Pixels: 28
mean: 0.9637609124183655
std: 0.1602856069803238,
max: 1.0
-----


In [ ]:
flattened_data_x, flattened_data_y = utils.get_flattened_data(raw_test_data)

In [ ]:
flattened_data_y.shape

In [ ]:
xx.shape

In [ ]:
raw_test_data = utils.preprocess_leaf_data(
    utils.load_leaf_test(configs['data']['FEMNIST_PATH']), min_num_samples=10
)

In [ ]:
raw_test_data.keys()

In [ ]:
import matplotlib.pyplot as plt
def draw_samples(data):
    list_keys = list(data.keys())
    data_np = dict()
    data_np['x'] = np.array(data[list_keys[0]]['x'], dtype = np.float32).reshape(-1, 28, 28)
    data_np['y'] = np.array(data[list_keys[0]]['y'], dtype = np.int64).reshape(-1, 1)
    figure = plt.figure(figsize=(8,8))
    for i in range(36):
        plt.subplot(6, 6, i + 1)
        plt.axis("off")
        plt.title(data_np['y'][i])
        plt.imshow(data_np['x'][i], cmap='gray')
        
draw_samples(raw_test_data)

In [ ]:
bb = utils.preprocess_leaf_data(raw_test_data)

In [ ]:
utils.dataset_info(raw_test_data)